In [31]:
import pandas as pd
import numpy as np
import sys, re, requests, collections
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 500)

Files related to teams include:
    1) data/teams.csv
    2) data/TeamCoaches.csv
    3) data/TeamConferences.csv
    4) data/TeamSpellings.csv
    

In [13]:
#Files related to teams

team_df = pd.read_csv('data/teams.csv', sep = ',', header = 0)
coaches_df = pd.read_csv('data/TeamCoaches.csv', sep = ',', header = 0)
conferences_df = pd.read_csv('data/TeamConferences.csv', sep = ',', header = 0)

name_mapping = collections.defaultdict(list)
with open('data/TeamSpellings.csv', 'rb') as f: #Some weird bytes at beginning of the file. 
    f.readline()
    out_line = [line.strip() for line in f.readlines()]
    for i in out_line:
        try:
            new_line = (i.decode('utf-8').split(","))
            name = new_line[0]
            teamId = new_line[1]
            name_mapping[teamId].append(name)
        except UnicodeDecodeError:
            pass

merged_df = coaches_df.merge(conferences_df, left_on = "TeamID", right_on = "TeamID")
merged_df = merged_df.merge(team_df, left_on = "TeamID", right_on = "TeamID")

The first outside data that I thought would be interesting to include would be the projected mock draft results of the year. Just as in any other sport, a superstar (or 2) can have a significant impact on a team's ability to survive March Madness. 

I used www.nbadraft.net to get draft results for each year we have data for (1985-present). Since we obviously won't have draft data for 2019, we will use mock draft projections as a good approximation. 

In [38]:
season_cresults = pd.read_csv('data/RegularSeasonCompactResults.csv')
season_dresults = pd.read_csv('data/RegularSeasonDetailedResults.csv')
tourney_cresults = pd.read_csv('data/NCAATourneyCompactResults.csv')
tourney_dresults = pd.read_csv('data/NCAATourneyDetailedResults.csv')
slots = pd.read_csv('data/NCAATourneySlots.csv')
seeds = pd.read_csv('data/NCAATourneySeeds.csv')

games = pd.concat((season_dresults, tourney_dresults), axis = 0, ignore_index = True)
games.reset_index(drop = True, inplace = True)

games["Point Differential"] = games["WScore"] - games["LScore"]
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['Point Differential Normal'] = games.apply(lambda r: r['Point Differential'] * -1 if r['Pred'] == 0. else r['Point Differential'], axis=1)


X = games[['ID','Team1Seed', 'Team2Seed', 'Team1MO', 'Team2MO', 'ScoreDiff','WScore', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]
y = games['Pred']
X_1 = games[['ID','Team1Seed', 'Team2Seed', 'Team1MO', 'Team2MO', 'ScoreDiff','WScore', 'LScore', 'WLoc']]

KeyError: "['ID' 'Team1Seed' 'Team2Seed' 'Team1MO' 'Team2MO' 'ScoreDiff'] not in index"

In [75]:
def getDraftResults(year, name_mapping):
    '''Works for Draft Years 1990-Present'''
    year = str(year)
    out_dict = {}
    url = "https://www.nbadraft.net/nba_final_draft/" + year

    html = requests.get(url)
    content = html.content
    soup = BeautifulSoup(content, 'html.parser')
    contentTable = soup.find_all('tbody')

    for i in contentTable:
        rows = i.find_all("tr")
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            outRow = ([ele for ele in cols if ele]) # Get rid of empty values
            try:
                pick = outRow[0]
                school = outRow[6].lower()
                schoolYear = outRow[7]
                if schoolYear != 'Intl.':
                    out_dict[pick] = name_mapping[school]
            except:
                pass
    return out_dict

draftpick_df = pd.DataFrame(merged_df["TeamID"])

getDraftResults(1991, name_mapping)

{}

In [78]:
url = 'https://stats.nba.com/draft/history/?Season=2018'

html = requests.get(url)
content = html.content
soup = BeautifulSoup(content, 'html.parser')
contentTable = soup.find_all('div', {'class' : 'nba-stat-table'})
print(contentTable)

[]
